#### read atom positions from .ion file

In [5]:
def parse_ion_file(file):
    positions = []
    with open(file, 'r') as f:
        for line in f:
            if not 'BLOCK POSITIONS' in line:
                pass
            else:
                positions = parse_position_section(f)
                break
    return(positions)
                
def parse_position_section(fs):
    pos = []
    for line in fs:
        if not 'END BLOCK POSITIONS' in line:
            coords = line.split()[1:4]
            coords = [float(c) for c in coords]
            pos.append(coords)
        else:
            break
    return(pos)

In [6]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/ion_step0.dat'
pos = parse_ion_file(file)
pos

[[-0.997005326, -1.09736017, -0.990137032],
 [0.994377532, 1.2895429, -0.992233496],
 [-0.993741539, 0.897105861, 0.984789156],
 [0.996369333, -1.08928866, 0.997581372]]

In [7]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/md_nve.ion'
parse_ion_file(file)

[[0.0, 0.0, 0.0], [0.5, 0.6, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5]]

#### read velocities from PROFESS

In [1]:
def parse_velocity_file(file):
    velocities = None
    with open(file, 'r') as f:
        for line in f:
            if not 'ION VELOCITIES (a.u.)' in line:
                pass
            else:
                velocities = parse_velocity_section(f)
                break
    return(velocities)
                
def parse_velocity_section(fs):
    velocities = []
    for line in fs:
        velocities_atom = line.split()
        velocities_atom = [float(v) for v in velocities_atom]
        velocities.append(velocities_atom)
    return(velocities)

In [2]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/vel_step0.dat'
vel = parse_velocity_file(file)
vel

[[-0.000205543441, -0.00024810832, 0.000107863699],
 [8.55258657e-05, -4.88537161e-05, 1.22627558e-05],
 [-5.66199546e-05, 0.000176480623, -0.000352144786],
 [0.000176640634, 0.000120470025, 0.000232021838]]

#### read forces from PROFESS

In [3]:
def parse_force_file(file):
    forces = []
    with open(file, 'r') as f:
        for line in f:
            if not 'TOTAL-FORCE' in line:
                pass
            else:
                f.readline()
                f.readline()
                forces = parse_force_section(f)
                break
    return(forces)

def parse_force_section(f):
    forces = []
    for line in f:
        if not '........' in line:
            forces_atom = line.split()[2:5]
            forces_atom = [float(for_a) for for_a in forces_atom]
            forces.append(forces_atom)
        else:
            break
    return(forces)

In [4]:
file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/force_step0.dat'
parse_force_file(file)

[[0.04849110742, 1.0197354373, -0.018138677031],
 [-0.036296786507, -2.0227648075, 0.0098813392233],
 [0.016044683227, 0.027908589553, 0.039171100695],
 [-0.02785916093, 0.97409727072, -0.030484722322]]